In [ ]:
# install dependencies: (use cu111 because colab has CUDA 11.1)
%pip install torch==1.10.0+cu111 torchvision==0.11.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
%pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html

# install mmdet for inference demo
%pip install mmdet

# install mmpose dependencies
%pip install -r requirements.txt

# install mmpose in develop mode
%pip install -e .

# Check Pytorch installation
import torch, torchvision

print('torch version:', torch.__version__, torch.cuda.is_available())
print('torchvision version:', torchvision.__version__)

# Check MMPose installation
import mmpose

print('mmpose version:', mmpose.__version__)

# Check mmcv installation
# from mmcv.ops import get_compiling_cuda_version, get_compiler_version

# print('cuda version:', get_compiling_cuda_version())
# print('compiler information:', get_compiler_version())

In [ ]:
from mmcv import Config
import mmcv

cfg = Config.fromfile('configs/animal/2d_kpt_sview_rgb_img/topdown_heatmap/ap10k/hrnet_w32_ap10k_256x256.py'
    #'./configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/hrnet_w32_coco_256x192.py'
)
cfg.data_root = 'data'
cfg.work_dir = 'work_dirs/hrnet_w32_acino_256x256'
cfg.data['train']['ann_file']=f'{cfg.data_root}/acino/annotations/acino_train.json'
cfg.data['train']['img_prefix']=f'{cfg.data_root}/acino/data/'
cfg.data['test']['ann_file']=f'{cfg.data_root}/acino/annotations/acino_test.json'
cfg.data['test']['img_prefix']=f'{cfg.data_root}/acino/data/'
cfg.data['val']['ann_file']=f'{cfg.data_root}/acino/annotations/acino_val.json'
cfg.data['val']['img_prefix']=f'{cfg.data_root}/acino/data/'
cfg.gpu_ids = range(1)
cfg.seed = 0
cfg.total_epochs=50

In [ ]:
print(cfg.pretty_text)

In [ ]:
from mmpose.datasets import build_dataset
from mmpose.models import build_posenet
from mmpose.apis import train_model

# build dataset
datasets = [build_dataset(cfg.data.train)]

# build model
model = build_posenet(cfg.model)

# create work_dir
mmcv.mkdir_or_exist(cfg.work_dir)

# train model
train_model(
    model, datasets, cfg, distributed=False, validate=True, meta=dict())

In [ ]:
from matplotlib import pprint
from mmpose.apis import (inference_top_down_pose_model, init_pose_model,
                         vis_pose_result, process_mmdet_results)
# from mmdet.apis import inference_detector, init_detector
from xtcocotools.coco import COCO
import os
# import cv2
local_runtime = False

# try:
#     from google.colab.patches import cv2_imshow  # for image visualization in colab
# except:
#     local_runtime = True
from mmpose.datasets import DatasetInfo

pose_checkpoint = 'work_dirs/hrnet_w32_acino_256x256/latest.pth'
json_file='tests/data/acino/testacino.json'
coco = COCO(json_file)
img_keys = list(coco.imgs.keys())

# initialize pose model
pose_model = init_pose_model(cfg, pose_checkpoint)

dataset = pose_model.cfg.data['test']['type']
dataset_info = pose_model.cfg.data['test'].get('dataset_info', None)
dataset_info = DatasetInfo(dataset_info)
return_heatmap=False
output_layer_names=None

for i in mmcv.track_iter_progress(range(len(img_keys))):
  # get bounding box annotations
  image_id = img_keys[i]
  #print("\nID:",image_id,"\n")
  image = coco.loadImgs(image_id)[0]
  #print("\nID:",image,"\n")
  image_name = os.path.join('tests/data/acino', image['file_name'])
  #print("\nID:",image_name,"\n")
  ann_ids = coco.getAnnIds(image_id)
  
  person_results = []
  for ann_id in ann_ids:
    person = {}
    ann = coco.anns[ann_id]
    
    # bbox format is 'xywh'
    person['bbox'] = ann['bbox']
    person_results.append(person)
    # print('\n')
    # print(person_results)
    # print('\n')

  pose_results, returned_outputs = inference_top_down_pose_model(
    pose_model,
    image_name,
    person_results=person_results,##
    bbox_thr=None,
    format='xywh',
    dataset=dataset,
    dataset_info=dataset_info,
    return_heatmap=return_heatmap,
    outputs=output_layer_names)
  
  os.makedirs('vis_result', exist_ok=True)
  out_file = os.path.join('vis_result', f'vis_as{image_id}.jpg')
  vis_result=vis_pose_result(pose_model,image_name,pose_results,radius=6,thickness=3,out_file=out_file)
  